In [1]:
import os

In [2]:
%pwd

'd:\\Proj_files\\Project_Machine_Learning\\Completed projects\\Text Summariation\\Text_Summarizer_Project_01\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Proj_files\\Project_Machine_Learning\\Completed projects\\Text Summariation\\Text_Summarizer_Project_01'